In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import numpy as np
train_data = pd.read_csv('train_data_with_samplefeatures.csv')
test_data = pd.read_csv('test_data_with_samplefeatures.csv')
train_data.shape
train_data.head(2)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,...,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,...,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,...,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.0,50.0,103.500000


In [2]:
train_data['winner_01'] = train_data.apply(lambda x: 1 if (x['team2']==x['winner']) else 0, axis=1)
train_data['toss_winner_01'] = np.where(train_data['toss winner']==train_data['team2'], 1, 0)
test_data['toss_winner_01'] = np.where(test_data['toss winner']==test_data['team2'], 1, 0)
train_data['toss_decision_01'] = np.where(train_data['toss decision']=='bat', 1, 0)
test_data['toss_decision_01'] = np.where(test_data['toss decision']=='bat', 1, 0)
X,y = train_data[['toss_winner_01','toss_decision_01','team_count_50runs_last15','team_winp_last5','team1only_avg_runs_last15','team1_winp_team2_last15','ground_avg_runs_last15']], train_data['winner_01']

In [3]:
X_test = test_data[X.columns.tolist()]
X_nans = X.isna().sum().reset_index()
X_nans[X_nans[0]!=0]

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans[X_test_nans[0]!=0]

,index,0
4,team1only_avg_runs_last15,21
6,ground_avg_runs_last15,53


,index,0
4,team1only_avg_runs_last15,7
6,ground_avg_runs_last15,10


In [4]:
X.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

C:\Users\singh\AppData\Local\Temp\ipykernel_15676\1698769224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)
C:\Users\singh\AppData\Local\Temp\ipykernel_15676\1698769224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace=True)


In [5]:
X_nans = X.isna().sum().reset_index()
X_nans[X_nans[0]!=0]

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans[X_test_nans[0]!=0]

,index,0


,index,0


In [6]:
X['toss_winner_01'].value_counts()
X['toss_decision_01'].value_counts()
X['team_count_50runs_last15'].describe()

1    620
0    328
Name: toss_winner_01, dtype: int64

0    617
1    331
Name: toss_decision_01, dtype: int64

count    948.000000
mean       1.202602
std        1.008793
min        0.083333
25%        0.686678
50%        1.000000
75%        1.400000
max       11.000000
Name: team_count_50runs_last15, dtype: float64

In [7]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 660.6 kB/s eta 0:02:31
   ---------------------------------------- 0.0/99.8 MB 660.6 kB/s eta 0:02:31
   ---------------------------------------- 0.0/99.8 MB 660.6 kB/s eta 0:02:31
   ---------------------------------------- 0.1/99.8 MB 393.8 kB/s eta 0:04:14
   ---------------------------------------- 0.1/99.8 MB 504.4 kB/s eta 0:03:18
   ---------------------------------------- 0.1/99.8 MB 504.4 kB/s eta 0:03:18
   ---------------------------------------- 0.2/99.8 MB 655.1 kB/s eta 0:02:32
   ---------------------------------------- 0.2/99.8 MB 654.9 kB/s eta 0:02:32
   ---------------------------------------- 0.4/99.8 MB 857.5 kB/s eta 0:01:56
   -----------

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve 
xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8, 12, 20, 30],
                  "n_estimators": [100, 200, 300],
                  "colsample_bytree": [0.5, 0.8, 1, 0.3, 0.4]}
X_train=X
y_train=y
randomcv_models = [
              
    ("Xgboost", XGBClassifier(), xgboost_params)
                   
                   ]
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5, 0.8, 1, 0.3,
                                                             0.4],
                                        'learning_rate': [0.1, 0.01],
                                        'max_depth': [5, 8, 12, 20, 30],
                                        'n_estimators': [100, 200, 300]},
                   verbose=2)

---------------- Best Params for Xgboost -------------------
{'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1}


In [17]:
models={
    
  
    "Xgboost":XGBClassifier(n_estimators=200,max_depth=5,learning_rate=0.01,
                           colsample_bytree=1)
}
X_train=X
y_train=y
for i in range(len(list(models))):
    model = list(models.values())[i]
    clf_gbm=model.fit(X_train, y_train) # Train model

    # Make predictions
    train_data['y_pred_01'] = clf_gbm.predict(X_train)
    test_data['y_pred_01'] = clf_gbm.predict(X_test)

    print(classification_report(y_train, clf_gbm.predict(X_train), labels=[0,1]))

              precision    recall  f1-score   support

           0       0.79      0.78      0.78       468
           1       0.79      0.80      0.79       480

    accuracy                           0.79       948
   macro avg       0.79      0.79      0.79       948
weighted avg       0.79      0.79      0.79       948



In [18]:
train_data['win_pred_score'] = clf_gbm.predict_proba(X_train)[:,1]
test_data['win_pred_score'] = clf_gbm.predict_proba(X_test)[:,1]

In [19]:
train_data['win_pred_score'] = np.where( (train_data['y_pred_01']==0), (1-train_data['win_pred_score']), train_data['win_pred_score'])
test_data['win_pred_score'] = np.where( (test_data['y_pred_01']==0), (1-test_data['win_pred_score']), test_data['win_pred_score'])

In [20]:
train_data['win_pred_team_id'] = np.where( (train_data['y_pred_01']==0), (train_data['team1_id']), train_data['team2_id'])
test_data['win_pred_team_id'] = np.where( (test_data['y_pred_01']==0), (test_data['team1_id']), test_data['team2_id'])

In [21]:
df_feat_importance = pd.DataFrame({'feat_name':X.columns.tolist(), 'model_feat_imp_train':clf_gbm.feature_importances_}).sort_values(by='model_feat_imp_train', ascending=False)\
                                                                                                                        .reset_index(drop=True).head(10)
df_feat_importance

,feat_name,model_feat_imp_train
0,team_count_50runs_last15,0.198907
1,ground_avg_runs_last15,0.168131
2,team_winp_last5,0.147117
3,team1only_avg_runs_last15,0.137129
4,team1_winp_team2_last15,0.131769
5,toss_winner_01,0.110973
6,toss_decision_01,0.105975


In [22]:
train_data['dataset_type'] = 'train'
test_data['dataset_type'] = 'r1'

In [23]:
algo_name = 'XGBoost'
is_ensemble = 'no'
n_trees = 200
depth = 5
lr = 0.01

In [24]:

df_file5 = pd.concat([test_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))], \
                     train_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))]])

renaming_dict = {}
for i,col in enumerate(list(df_feat_importance['feat_name'].head(10))):
    renaming_dict[col] = f'indep_feat_id{i+1}'
df_file5.rename(columns=renaming_dict, inplace=True)

for i in range(1,11):
    if f'indep_feat_id{i}' not in df_file5.columns:
        df_file5[f'indep_feat_id{i}'] = np.nan

df_file5['train_algorithm'] = algo_name
df_file5['is_ensemble'] = is_ensemble
df_file5['train_hps_trees'] = n_trees
df_file5['train_hps_depth'] = depth
df_file5['train_hps_lr'] = lr

In [25]:
df_file5.shape
df_file5.head()

(1219, 19)

,match id,dataset_type,win_pred_team_id,win_pred_score,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr
0,9250275,r1,90,0.763316,0.000000,NaN,0.019608,NaN,0.00,1,0,NaN,NaN,NaN,XGBoost,no,200,5,0.01
1,9262189,r1,36098,0.580087,0.615385,153.500,0.344262,151.285714,66.67,1,0,NaN,NaN,NaN,XGBoost,no,200,5,0.01
2,9128776,r1,48334,0.573506,0.842105,179.625,0.753086,171.066667,100.00,0,1,NaN,NaN,NaN,XGBoost,no,200,5,0.01
3,9586919,r1,36112,0.584541,0.285714,155.500,1.487805,166.000000,75.00,1,0,NaN,NaN,NaN,XGBoost,no,200,5,0.01
4,9128538,r1,30414,0.652463,2.375000,164.125,0.310330,169.933333,0.00,1,0,NaN,NaN,NaN,XGBoost,no,200,5,0.01


In [26]:
feature_desc = {'team_count_50runs_last15':'Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games',\
               'team_winp_last5':'Ratio of team1\'s win % to team2\'s win % in last 5 games',\
               'ground_avg_runs_last15':'average runs scored in the ground in last 15 games',\
               'team1_winp_team2_last15':'Team1\'s win percentage against Team2 in last 15 games',\
               'team1only_avg_runs_last15':'team1\'s avg inning runs in last 15 games',\
                
               'season_num' : 'Numerical form of season. Takes 1 for oldest season and increases for latest seasons.',\
               'toss_winner_01' : 'Toss winner to numerical - 1 if team2 wins, else 0',\
               'toss_decision_01' : 'Toss decision - categorical - 1 if winner bats, 0 otherwise'}

In [27]:
df_file6 = df_feat_importance
df_file6['feat_id'] = [i+1 for i in df_file6.index]
df_file6['feat_rank_train'] = [i+1 for i in df_file6.index]
df_file6 = df_file6.set_index('feat_id')
df_file6['feat_description'] = df_file6['feat_name'].map(feature_desc)

In [30]:

df_file6.to_csv('file7.csv')